In [5]:
import plotto
%matplotlib inline
import imp
imp.reload(plotto)
from collections import OrderedDict as odict
class O(object):
    def __init__(self, **kw):
            for k in kw:
                    self.__dict__[k]=kw[k]

In [6]:
meneghetti = plotto.ObservativeTable('data/meneghetti2014.t')
meneghetti.M200c

<Quantity([8.5200004e+14 9.4300007e+14 7.0000001e+14 9.9799994e+14 6.1200002e+14
 8.5000000e+14 1.0410000e+15 6.1900001e+14 1.1500000e+15 9.0000000e+14
 6.9200003e+14 7.5000001e+14 8.0500002e+14 8.6200000e+14 7.3100001e+14
 1.1470000e+15 6.0899999e+14 5.7100000e+14 7.0000001e+14], 'Msun')>

In [7]:



interp_observations = odict({"Bullock et al 2001":O(xrange=[11,13.5],  c8e14=5.12, B=-0.13) ,
             "Neto et al. 2007": O(xrange=[11,15], c8e14=3.46, B=-0.11),
             "Neto et al. 2007 only relaxed": O(xrange=[11,15], c8e14=5.27, B=-0.11),
             "Bhattacharya et al. 2013":O(xrange=[11,14],  c8e14=3.8, B=-0.08),
             "Dutton & Maccio 2014": O(xrange=[11,14], c8e14=3.94, B=-0.1),
             "Dutton & Maccio 2014 only relaxed": O(xrange=[11,14], c8e14=4.42, B=-0.1),
             "Mantz et. al 2015": O(xrange=[12,14], c8e14=3.12, B=-0.32),
             "Groener et al 2016 WL": O(xrange=[14,15], c8e14=7., B=-0.42),
             "Groener et al 2016 WL+SL": O(xrange=[14,15], c8e14=5., B=-0.43),
             "Kyplin et al. 2016": O(xrange=[14,15], c8e14=3.8, B=-0.12)})
             


In [8]:
pratt = plotto.ObservativeTable('data/Pratt_et_al_2016.txt')



In [9]:
import numpy as np
import matplotlib.pyplot as plt

rsfac = 0.46556

NFW = lambda r,rho0,rs: rho0/((r/rs)*(1.+r/rs)**2.)
#xs = np.logspace(-1,np.log10(.46),200)
#profile = NFW(xs, 1./rsfac, 10.)

NFW(rsfac,  10., 1.)

10.000394537089177

In [10]:
#xs[-1], profile[-1]

In [11]:
def sc(c0,a):
    return lambda x: c0*(x/8.e14)**a

def set_Mc_lims(a):
    a.set_xscale('log')
    a.set_xlim([1e11,2e15])
    a.set_ylim([1,20])
    a.set_yscale('log')
    a.set_xlabel("M200c [Msun]")
    a.set_ylabel("c200c ")
    a.legend()

In [22]:
rsfac = 0.46556
NFW = lambda r,rho0,rs: rho0/((r/rs)*(1.+r/rs)**2.)




f_NFW = lambda c:(np.log(1.+c)-c/(1.+c))

def cdelta1(delta1,delta2,cdelta2):
    return lambda cdelta1: cdelta2 * (  (delta2/delta1)*(f_NFW(cdelta1)/f_NFW(cdelta2))  )**(1./3.)


def banach_caccioppoli(f,x0,accuracy=0.00001): 
    """ this function solves equation of the kind x=f(x) by iterations, given an initial value x=x0"""
    condition=True
    x1=x0
    steps=0
    while condition:
        
        steps+=1
        x2 = f(x1)
        error = np.abs(x2-x1)/x2
        condition = np.all(error>accuracy)
        x1=x2
    return x2, steps,error

def convert_concentration(new_delta, old_delta, old_concentration):
    new_concentration,steps,error = banach_caccioppoli( cdelta1(new_delta, old_delta,  old_concentration), old_concentration)
    return new_concentration

def convert_mass(new_delta, old_delta,  new_concentration, old_concentration, old_mass):
    return old_mass*(new_delta/old_delta)*(new_concentration/old_concentration)**3

def convert_mass_and_concentration(new_delta, old_delta, old_concentration, old_mass):
    new_concentration,steps,error = banach_caccioppoli( cdelta1(new_delta, old_delta,  old_concentration), old_concentration)
    new_mass = old_mass*(new_delta/old_delta)*(new_concentration/old_concentration)**3
    return new_mass, new_concentration




In [23]:

pratt.c200 = convert_concentration(200., 500., pratt.c500)
pratt.M200 = convert_mass(200., 500., pratt.c200, pratt.c500,  pratt.M500)